<a href="https://colab.research.google.com/github/Anjani5G/Extremum_finder/blob/main/extremum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

## Making a suitable dataset

In [77]:
# generating cummulative monotonic functions
for i in range(100):
  x = np.linspace(0, 10+np.random.normal(0, 2), 100)
  y = np.random.uniform(0,1,100)
  df = pd.DataFrame(data = {'x':x, 'random':y})
  df['y'] = df['random'].cumsum()
  if np.random.randint(low=0,high=2) == 1:
    df['y'] = np.flip(df['y'].to_xarray())
  plt.plot(df.x, df.y)
  plt.savefig(f'data/data_{i}N.png')
  plt.clf()

# generating piece-wise linear monotonic functions
for i in range(100):
  x = np.linspace(0, 10 + np.random.normal(0, 2), 100)
  y = [i + np.random.uniform(0,1) for i in x]
  df = pd.DataFrame(data = {'x':x, 'y':y})
  if np.random.randint(low=0,high=2) == 1:
    df['y'] = np.flip(df['y'].to_xarray())
  plt.plot(df.x, df.y)
  plt.savefig(f'data/data_{i}Y.png')
  plt.clf()

<Figure size 640x480 with 0 Axes>

## Loading and training the model

In [86]:
# Define the directory containing the images
data_dir = 'data'

# Initialize lists to hold the data and labels
image_data = []
labels = []

# Iterate over the files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png'):
        # Load the image
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
        img_resized = cv2.resize(img, (640, 480))  # Resize the image to 640x480 pixels
        img_array = img_resized / 255.0  # Normalize the image to [0, 1]

        # Append the image data and label to the lists
        image_data.append(img_array)
        label = filename.split('_')[-1][1]  # Extract label ('N' or 'Y')
        labels.append(label)

# Convert lists to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Reshape the image data to include a channel dimension for grayscale
image_data = image_data.reshape(-1, 480, 640, 1)

# Print the shape of the image data and labels to verify
print("Image data shape:", image_data.shape)
print("Labels shape:", labels.shape)

# Encode the labels as integers (0 for 'N', 1 for 'Y')
labels_encoded = np.array([0 if label == 'N' else 1 for label in labels])

# ---

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels_encoded, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# ---

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# -------------

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# ---------

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Image data shape: (200, 480, 640, 1)
Labels shape: (200,)
X_train shape: (160, 480, 640, 1)
X_test shape: (40, 480, 640, 1)
y_train shape: (160,)
y_test shape: (40,)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 478, 638, 32)      320       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 239, 319, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 118, 158, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D) 